In [2]:
# Importer les modules nécessaires
import os
import sys
import pandas as pd
from google.cloud import storage

# Ajouter le chemin de base de ton projet au PYTHONPATH
sys.path.append("/root/Trustia/Cicada-binance")

# Importer le EnvController pour la gestion de l'environnement
from src.commons.env_manager.env_controller import EnvController

# Initialiser le EnvController pour charger les configurations
env_controller = EnvController(config_base_path="/root/Trustia/Cicada-binance/config")

# Récupérer le nom du bucket depuis les configurations YAML
bucket_name = env_controller.get_yaml_config('Ta-lib', 'bucket')
print(f"Nom du bucket: {bucket_name}")

# Créer un client Google Cloud Storage
client = storage.Client()

# Accéder au bucket GCS
bucket = client.bucket(bucket_name)


Loading shared env from: /root/Trustia/Cicada-binance/config/shared/.env.shared
Loading environment-specific env from: /root/Trustia/Cicada-binance/config/development/secrets.env
Loading shared YAML config from: /root/Trustia/Cicada-binance/config/shared/db-config.yaml
Loading environment-specific YAML config from: /root/Trustia/Cicada-binance/config/development/app-config.yaml
Loading YAML file: /root/Trustia/Cicada-binance/config/development/app-config.yaml
Nom du bucket: production-trustia-raw-data


In [3]:
def load_gcs_file_to_dataframe(bucket_name, file_path, file_format='parquet'):
    """
    Télécharge un fichier à partir de Google Cloud Storage et le charge dans un DataFrame.

    :param bucket_name: Le nom du bucket GCS.
    :param file_path: Le chemin du fichier dans le bucket.
    :param file_format: Le format du fichier ('parquet', 'csv', etc.).
    :return: DataFrame contenant les données du fichier.
    """
    try:
        # Créer un client GCS
        client = storage.Client()
        bucket = client.bucket(bucket_name)
        
        # Récupérer le blob (fichier) depuis le bucket
        blob = bucket.blob(file_path)
        
        # Créer un fichier temporaire local
        temp_file_path = '/tmp/temp_file.' + file_format
        
        # Télécharger le fichier depuis GCS
        blob.download_to_filename(temp_file_path)
        print(f"Fichier téléchargé depuis GCS : {temp_file_path}")
        
        # Charger le fichier dans un DataFrame
        if file_format == 'parquet':
            df = pd.read_parquet(temp_file_path)
        elif file_format == 'csv':
            df = pd.read_csv(temp_file_path)
        else:
            raise ValueError(f"Format de fichier non supporté : {file_format}")
        
        # Retourner le DataFrame
        return df
    
    except Exception as e:
        print(f"Erreur lors du téléchargement ou du chargement du fichier : {e}")
        return None


In [ ]:
# Spécifier le chemin du fichier dans le bucket (sans le nom du bucket)
gcs_file_path = "Transformed/cryptos/ADAUSDT/futures/bars/time-bars/1h/data_2020_01_31_2024_10_08.parquet"  # Remplace par le chemin correct du fichier

# Charger le fichier GCS dans un DataFrame
df = load_gcs_file_to_dataframe(bucket_name, gcs_file_path, file_format='parquet')

# Afficher le DataFrame
if df is not None:
    print("DataFrame chargé avec succès :")
    display(df)
else:
    print("Le DataFrame n'a pas pu être chargé.")

Erreur lors du téléchargement ou du chargement du fichier : 404 GET https://storage.googleapis.com/download/storage/v1/b/production-trustia-raw-data/o/Transformed%2Fcryptos%2FADAUSDT%2Ffutures%2Fbars%2Ftime-bars%2F1h%2Fdata_2012_01_31_2024_10_08.parquet?alt=media: No such object: production-trustia-raw-data/Transformed/cryptos/ADAUSDT/futures/bars/time-bars/1h/data_2012_01_31_2024_10_08.parquet: ('Request failed with status code', 404, 'Expected one of', <HTTPStatus.OK: 200>, <HTTPStatus.PARTIAL_CONTENT: 206>)
Le DataFrame n'a pas pu être chargé.


In [13]:
df.columns

Index(['timestamp', 'open', 'high', 'low', 'close', 'volume', 'EMA_20',
       'MACD_12_26_9', 'MACDh_12_26_9', 'MACDs_12_26_9', 'RSI_14'],
      dtype='object')

DEPRECATED

In [ ]:
def read_parquet_with_pyarrow(self, file_path):
    try:
        table = pq.read_table(file_path)
        df = table.to_pandas()
        df.columns = [
            'open_time', 'open', 'high', 'low', 'close', 'volume',
            'close_time', 'quote_volume', 'count', 'taker_buy_volume',
            'taker_buy_quote_volume', 'ignore'
        ]
        return df
    except Exception as e:
        self.logger.log_error(f"Error reading parquet file with pyarrow: {e}")
        return pd.DataFrame()

In [7]:
# Spécifier le chemin du fichier dans le bucket (sans le nom du bucket)
gcs_file_path = "Raw/binance-data-vision/historical/BNBUSDT/spot/klines/1h/2024/10/16/data.parquet"  # Remplace par le chemin correct du fichier

# Charger le fichier GCS dans un DataFrame
df = load_gcs_file_to_dataframe(bucket_name, gcs_file_path, file_format='parquet')

# Afficher le DataFrame
if df is not None:
    print("DataFrame chargé avec succès :")
    display(df)
else:
    print("Le DataFrame n'a pas pu être chargé.")


Fichier téléchargé depuis GCS : /tmp/temp_file.parquet
DataFrame chargé avec succès :


,1729036800000,594.60000000,595.20000000,592.70000000,593.00000000,5349.04000000,1729040399999,3177791.59820000,20161,2828.54200000,1680396.76870000,0
0,1729040400000,593.0,596.9,591.7,595.9,7483.837,1729043999999,4.451768e+06,32057,3904.872,2.323116e+06,0
1,1729044000000,595.9,597.4,595.0,595.8,5515.409,1729047599999,3.288000e+06,21996,3084.930,1.839163e+06,0
2,1729047600000,595.9,596.3,593.3,593.9,6029.503,1729051199999,3.586168e+06,19292,3151.599,1.874314e+06,0
3,1729051200000,593.9,595.2,589.7,591.4,12690.782,1729054799999,7.509178e+06,28567,4308.481,2.549327e+06,0
4,1729054800000,591.4,593.9,590.8,592.9,5549.257,1729058399999,3.288479e+06,19417,3812.459,2.259460e+06,0
5,1729058400000,592.9,593.8,590.2,591.8,7640.028,1729061999999,4.519972e+06,31396,3332.147,1.971667e+06,0
6,1729062000000,591.8,592.5,589.7,591.0,7792.389,1729065599999,4.605405e+06,28330,4048.683,2.393138e+06,0
7,1729065600000,591.1,593.6,590.6,592.0,8245.886,1729069199999,4.884115e+06,30981,4421.229,2.619219e+06,0
8,1729069200000,592.1,595.3,591.9,594.6,6671.401,1729072799999,3.959864e+06,22862,3782.672,2.245456e+06,0
9,1729072800000,594.7,597.2,593.3,595.9,21628.076,1729076399999,1.287766e+07,38686,12110.703,7.209129e+06,0
